In [ ]:
from __future__ import print_function
import os.path
import dalmatian as dm
import pandas as pd

In [ ]:
namespace="broad-genomics-delivery"
workspace="Getz_IBM_CellLines_RNASeqData"
refnamespace="broad-firecloud-ccle"
refworkspace="DepMap_hg38_RNAseq"
source="ibm"
samplesetname="CCLE_19Q3interim"

In [ ]:
wm = dm.WorkspaceManager(namespace, workspace)
refwm = dm.WorkspaceManager(refnamespace, refworkspace)

# Generate sample set from new samples

In [ ]:
refsamples = refwm.get_samples()
refids = refsamples['participant'].tolist()
refids = [val[val.index('ACH'):] for val in refids]
refsamples

In [ ]:
samples = wm.get_samples()
samples = samples[samples['individual_alias'].str.contains('ACH')][~samples['individual_alias'].str.slice(0,10).isin(refids)]
samples['individual_alias']

In [ ]:
for ind, val in samples.iterrows():
    refsamples = refsamples.append(pd.Series({
        "CCLE_name": val['sample_alias'],
        "WES_bai": val['crai_or_bai_path'],
        "WES_bam": val['cram_or_bam_path'],
        "Source": source,
        "participant":val['individual_alias'][:10],
    },name=source+'_'+val['individual_alias'][:10]))

In [ ]:
refsamples.shape

In [ ]:
refwm.upload_samples(refsamples)

In [ ]:
sample_ids = [source+'_'+i for i in samples['individual_alias'].str.slice(0,10).tolist()]
sample_ids

In [ ]:
refwm.update_sample_set(sample_set_id=samplesetname,sample_ids=sample_ids)

# run the pipeline

In [ ]:
samtofastq = refwm.get_config("samtofastq_v1-0_BETA_cfg")
samtofastq

In [ ]:
samtofastq['samtofastq_workflow.samtofastq.input_bam_cram']= 'this.WES_bam'
refwm.update_config(samtofastq)
submission_id = refwm.create_submission(samtofastq['name'], samplesetname,'sample_set',expression='this.samples')
while wm.get_submission(submission_id)['status'] not in {'Done', 'Aborted'}:
    time.sleep(60)
assert wm.get_submission(submission_id)['status'] !='Aborted'

In [ ]:
star = refwm.get_config("star_v1-0_BETA_cfg")
star

In [ ]:
refwm.update_config(star)
submission_id = refwm.create_submission(star['name'], samplesetname,'sample_set',expression='this.samples')
while wm.get_submission(submission_id)['status'] not in {'Done', 'Aborted'}:
    time.sleep(60)
assert wm.get_submission(submission_id)['status'] !='Aborted'

In [ ]:
rsem = refwm.get_config("rsem_v1-0_BETA_cfg")
rsem

In [ ]:
refwm.update_config(rsem)
submission_id1 = refwm.create_submission(rsem['name'], samplesetname,'sample_set',expression='this.samples')

In [ ]:
fusion = refwm.get_config("hg38_STAR_fusion")
fusion

In [ ]:
fusion['inputs']['trinity_cleaned.samtofastq.input_bam'] = 'this.WES_bam'
refwm.update_config(fusion)
submission_id2 = refwm.create_submission(fusion['name'], samplesetname,'sample_set',expression='this.samples')

In [ ]:
wait_for_submission([submission_id1,submission_id2])

In [ ]:
import time
def wait_for_submission(submissions):
    failed_submission = []
    if type(submissions) is type(""):
        submissions = [submissions]
    for submission_id in submissions:
        if not wm.get_submission(submission_id)['workflows']:
            while wm.get_submission(submission_id)['status'] not in {'Done', 'Aborted'}:
                time.sleep(60)
        else:
            for i in wm.get_submission(submission_id)["workflows"]:
                while i['status'] not in {'Done', 'Aborted'}:
                    time.sleep(60)
                if i["status"]=='Failed':
                    print(newsample.loc[i["workflowEntity"]["entityName"]]["attr_sex"])
                    print(i["workflowEntity"]["entityName"])
                    failed_submission.append(i["workflowEntity"]["entityName"])
    # print and return well formated data

In [ ]:
a = []
type(a) is str

In [ ]:
wm.get_submission(submission_id2)